# Primeira entrega

A primeira entrega consiste no carregamento dos dados e na construção de um classificador para os dados do EEG Dataset. 


### Bibliotecas, seleção dos eletrodos e estrutura dos dados ###

Foram utilizadas as bibliotecas: os, numpy, matplotlib, mne e sklearn.

Foi feito uma seleção de eletrodos que achamos ser relevantes, sendo eles: **F8**, **FT8**, **TP8**, **P8**, **FP1**, **AF** e **AF7**.

Estruturas para os dados de treino e teste foram definidas.


In [10]:
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne import set_eeg_reference as car
from mne.time_frequency import psd_welch as pw
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.decomposition import PCA, FastICA

NUMERO_DE_LEITURAS_POR_ELETRODO_POR_TRIAL = 256

#Eletrodos
ch_names =  [
            'FP1','FP2','F7','F8','AF1','AF2','FZ','F4','F3','FC6','FC5','FC2','FC1',
            'T8','T7','CZ','C3','C4','CP5','CP6','CP1','CP2','P3','P4','PZ','P8','P7','PO2','PO1',
            'O2','O1','X','AF7','AF8','F5','F6','FT7','FT8','FPZ','FC4','FC3','C6','C5','F2','F1',
            'TP8','TP7','AFZ','CP3','CP4','P5','P6','C1','C2','PO7','PO8','FCZ','POZ','OZ','P2','P1','CPZ','nd','Y'
            ]

# ch_relevantes = ['TP8'] 
# ch_relevantes = ['AF7']        
# ch_relevantes = ['F8', 'FT8', 'TP8', 'P8']
# ch_relevantes = ['FP1', 'AF', 'AF7']
ch_relevantes = ['F8', 'FT8', 'TP8', 'P8', 'FP1', 'AF', 'AF7']

#TP8 = 45
#AF7 = 32

data_treino =  {
            'oa' : [],
            'oc' : [],

            'ma' : [],
            'mc' : [],

            'na' : [],
            'nc' : []
        }


data_teste = []

caracteristicas_treino =  {
    'oa' : [],
    'oc' : [],

    'ma' : [],
    'mc' : [],

    'na' : [],
    'nc' : []
}

caracteristicas_teste = {
    'o' : [],
    'm' : [],
    'n' : []
}

target_teste = {
    'o' : [],
    'm' : [],
    'n' : []
}


### Carregamento dos dados

Foi feito o carregamento dos dados e uma divisão dos dados de **treino** e **teste**.

O **treino** possui 20 amostras cada uma com 30 trials contendo 64 canais. Para o **teste** as quantidade de amostras, trials e canais se mantem.

In [11]:
pastas = listdir('teste')
for pasta in pastas:
    grupo = pasta[3]
    naosei = {'o' : [], 'm' : [], 'n' : [], 'grupo' : grupo}
    arquivos = listdir('teste/' + pasta)
    for arquivo in arquivos:
        parar = False
        arq = open('teste/' + pasta + '/' + arquivo)
        conteudo = arq.read().split('\n')
        experimento = conteudo[3][5]
        i = 4
        valor = list()
        canais = list()
        while not parar:
            if conteudo[i] == '':                           # se chegou na última linha do arquivo
                naosei[experimento].append(canais)          # adiciona os dados do arquivo atual na lista
                parar = True                                # parar
            else:
                if conteudo[i][0] == '#':                   # se for linha de identificação de eletrodo
                    valor = list()                          # limpa a lista de valores
                else:                                       # se for linha com valor
                    valor.append(conteudo[i].split(' ')[3]) # guarda o valor lido
                    if len(valor) == NUMERO_DE_LEITURAS_POR_ELETRODO_POR_TRIAL: # se ja leu os 256 valores
                        canais.append(valor)                # adiciona os valores de canal atual
            i += 1
    data_teste.append(naosei)
pastas = listdir('treino')
for pasta in pastas:
    arquivos = listdir('treino/' + pasta)
    for arquivo in arquivos:
        parar = False
        arq = open('treino/' + pasta + '/' + arquivo)
        conteudo = arq.read().split('\n')
        experimento = conteudo[3][5]
        grupo = pasta[3]
        i = 4
        valor = list()
        canais = list()
        while not parar:
            if conteudo[i] == '':                           # se chegou na última linha do arquivo
                data_treino[experimento+grupo].append(canais)      # adiciona os dados do arquivo atual na lista
                parar = True                                # parar
            else:
                if conteudo[i][0] == '#':                   # se for linha de identificação de eletrodo
                    valor = list()                          # limpa a lista de valores
                else:                                       # se for linha com valor
                    valor.append(conteudo[i].split(' ')[3]) # guarda o valor lido
                    if len(valor) == NUMERO_DE_LEITURAS_POR_ELETRODO_POR_TRIAL: # se ja leu os 256 valores
                        canais.append(valor)                # adiciona os valores de canal atual
            i += 1

### Pré-processamento

A variável "data_treino" possui todos os dados do conjunto de treino.

Os canais "X", "nd" e "Y" foram excluidos dos dados.

Foi aplicado o filtro not no objeto "raw" para que a frequência em 60Hz e seus harmônicos sejam removidos. Aplicando também um filtro passa faixa no intervalo de 5Hz a 50Hz.

Aplicando o método "car()" nos canais que foram selecionados como relevantes, os demais canais têm seus valores aproximados aos canais escolhidos.

Estes procedimentos são igualmente aplicados ao conjunto de teste.

In [12]:
info = mne.create_info(ch_names=ch_names, sfreq=256, ch_types=['eeg'] * 64)

for tipo in data_treino:
    for i in range(len(data_treino[tipo])):
        raw = mne.io.RawArray(data_treino[tipo][i], info, verbose=False)
        raw.drop_channels(['X', 'nd', 'Y'])
        montage = mne.channels.read_montage('standard_1020')
        raw.set_montage(montage)
        raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
        raw.filter(5., 50., fir_design='firwin')
        # raw.plot_psd(area_mode='range')
        
        raw.set_eeg_reference(ref_channels='average', verbose=False)
        psds, freqs = pw(raw, verbose=False) 
        media = []
        
        for canal in psds:
            media.append(canal.mean())
        
        caracteristicas_treino[tipo].append(media)
        
        #ica = FastICA(n_components=1)
        #processado = ica.fit_transform(raw.get_data().T).T
        #caracteristicas_treino[tipo].append(processado[0])
        
        # pca = PCA(n_components=1)
        # processado = pca.fit_transform(raw.get_data().T).T
        # caracteristicas_treino[tipo].append(processado[0])

        # processado = raw.get_data()[AF7]
        # caracteristicas_treino[tipo].append(processado)

for i in range(len(data_teste)):
    for tipo in data_teste[i]:
        if tipo != 'grupo':
            for j in range(len(data_teste[i][tipo])):

                raw = mne.io.RawArray(data_teste[i][tipo][j], info, verbose=False)
                
                raw.drop_channels(['X', 'nd', 'Y'])
                montage = mne.channels.read_montage('standard_1020')
                raw.set_montage(montage)
                raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
                raw.filter(5., 50., fir_design='firwin')
                # raw.plot_psd(area_mode='range')
                
                raw.set_eeg_reference(ref_channels='average', verbose=False)
                psds, freqs = pw(raw, verbose=False) 
                media = []
                
                for canal in psds:
                    media.append(canal.mean())
                
                caracteristicas_teste[tipo].append(media)
                
                #ica = FastICA(n_components=1)
                #processado = ica.fit_transform(raw.get_data().T).T
                #caracteristicas_teste[tipo].append(processado[0])

                # pca = PCA(n_components=1)
                # processado = pca.fit_transform(raw.get_data().T).T
                # caracteristicas_teste[tipo].append(processado[0])

                # processado = raw.get_data()[AF7]
                # caracteristicas_teste[tipo].append(processado)

                target_teste[tipo].append(data_teste[i]['grupo'])

print()

#data_treino['oa'].shape() = 100, 64, 256
    
#print(data_treino['oa'][99])    
    
#print(data_treino['oa'].shape())
#print(data_teste['o'].shape())

<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than

<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than

<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than

<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than

<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:10: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:9: RuntimeWarning: filter_length (1691) is longer than

...

<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer th

<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer t

<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer th

<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer th

<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')
<ipython-input-12-52edd6b91536>:43: RuntimeWarning: filter_length (1691) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(np.arange(60, 121, 60), fir_design='firwin')
<ipython-input-12-52edd6b91536>:44: RuntimeWarning: filter_length (423) is longer than the signal (256), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(5., 50., fir_design='firwin')


### Normalização das características ###

Foi feito a normalização das características do treino e teste utilizando o método "fit()" da biblioteca Sklearn.


In [13]:
# normalizacao
for tipo in caracteristicas_treino:
    x = PowerTransformer().fit_transform(caracteristicas_treino[tipo])
                                                                            
    caracteristicas_treino[tipo] = x
    #ss = StandardScaler()
    #caracteristicas_treino[tipo] = ss.fit_transform(np.array(caracteristicas_treino[tipo]))
    #ss.fit(caracteristicas_treino[tipo])
    #caracteristicas_treino[tipo] = ss.transform(caracteristicas_treino[tipo])

for tipo in caracteristicas_teste:
    x = PowerTransformer().fit_transform(caracteristicas_teste[tipo])
    caracteristicas_teste[tipo] = x
    #ss = StandardScaler()
    #caracteristicas_teste[tipo] = ss.fit_transform(np.array(caracteristicas_teste[tipo]))
    #ss.fit(caracteristicas_teste[tipo])
    #caracteristicas_teste[tipo] = ss.transform(caracteristicas_teste[tipo])


### Classificação ###
Abaixo é mostrado o resultado da classificação do experimento com 1 objeto, 2 objetos relacionados e 2 objetos distintos utilizando o SVM, o KNN, o NN e a árvore de decisão.

In [14]:
#classificacao
X_train = list(caracteristicas_treino['oa']) + list(caracteristicas_treino['oc'])
Y_train = ['a'] * len(caracteristicas_treino['oa']) + ['c'] * len(caracteristicas_treino['oc'])
X_test = caracteristicas_teste['o']
Y_test = target_teste['o']

gamma = 0.01
classificador = SVC(gamma=gamma)
classificador.fit(X_train, Y_train)
print('SVM - Experimento com 1 objeto:', classificador.score(X_test, Y_test) * 100, '%')

k = 5
classificador = neighbors.KNeighborsClassifier(k, 'distance')
classificador.fit(X_train, Y_train)
print('KNN - Experimento com 1 objeto:', classificador.score(X_test, Y_test) * 100, '%')

'''
sub = 8
classificador = MLPClassifier(hidden_layer_sizes=(sub), random_state=0)
classificador.fit(X_train, Y_train)
print('NN - Experimento com 1 objeto:', classificador.score(X_test, Y_test) * 100, '%')
'''
classificador = tree.DecisionTreeClassifier()
classificador.fit(X_train, Y_train)
print('TREE - Experimento com 1 objeto:', classificador.score(X_test, Y_test) * 100, '%\n')

##################################################################################################

X_train = list(caracteristicas_treino['ma']) + list(caracteristicas_treino['mc'])
Y_train = ['a'] * len(caracteristicas_treino['ma']) + ['c'] * len(caracteristicas_treino['mc'])
X_test = caracteristicas_teste['m']
Y_test = target_teste['m']

gamma = 0.01
classificador = SVC(gamma=gamma)
classificador.fit(X_train, Y_train)
print('SVM - Experimento com 2 objetos relacionados:', classificador.score(X_test, Y_test) * 100, '%')

k = 5
classificador = neighbors.KNeighborsClassifier(k, 'distance')
classificador.fit(X_train, Y_train)
print('KNN - Experimento com 2 objetos relacionados:', classificador.score(X_test, Y_test) * 100, '%')
'''
sub = 8
classificador = MLPClassifier(hidden_layer_sizes=(sub), random_state=0)
classificador.fit(X_train, Y_train)
print('NN - Experimento com 2 objetos relacionados:', classificador.score(X_test, Y_test) * 100, '%')
'''
classificador = tree.DecisionTreeClassifier()
classificador.fit(X_train, Y_train)
print('TREE - Experimento com 2 objetos relacionados:', classificador.score(X_test, Y_test) * 100, '%\n')

##################################################################################################

X_train = list(caracteristicas_treino['na']) + list(caracteristicas_treino['nc'])
Y_train = ['a'] * len(caracteristicas_treino['na']) + ['c'] * len(caracteristicas_treino['nc'])
X_test = caracteristicas_teste['n']
Y_test = target_teste['n']

gamma = 0.01
classificador = SVC(gamma=gamma)
classificador.fit(X_train, Y_train)
print('SVM - Experimento com 2 objetos distintos:', classificador.score(X_test, Y_test) * 100, '%')

k = 10
classificador = neighbors.KNeighborsClassifier(k, 'distance')
classificador.fit(X_train, Y_train)
print('KNN - Experimento com 2 objetos distintos:', classificador.score(X_test, Y_test) * 100, '%')
'''
sub = 8
classificador = MLPClassifier(hidden_layer_sizes=(sub), random_state=0)
classificador.fit(X_train, Y_train)
print('NN - Experimento com 2 objetos distintos:', classificador.score(X_test, Y_test) * 100, '%')
'''
classificador = tree.DecisionTreeClassifier()
classificador.fit(X_train, Y_train)
print('TREE - Experimento com 2 objetos distintos:', classificador.score(X_test, Y_test) * 100, '%')

SVM - Experimento com 1 objeto: 81.0 %
KNN - Experimento com 1 objeto: 84.0 %
TREE - Experimento com 1 objeto: 55.00000000000001 %

SVM - Experimento com 2 objetos relacionados: 84.0 %
KNN - Experimento com 2 objetos relacionados: 82.5 %
TREE - Experimento com 2 objetos relacionados: 59.0 %

SVM - Experimento com 2 objetos distintos: 75.0 %
KNN - Experimento com 2 objetos distintos: 81.0 %
TREE - Experimento com 2 objetos distintos: 62.0 %
